In [1]:
import ee, geetools

ee.Initialize()

In [8]:
d = ee.DateRange('2020-01-01', '2020-01-31').geetools.toList(1, 'day')
ee.DateRange(d.get(0)).start().format("YYYY-MM-DD").getInfo()

'2020-01-01'